<a href="https://colab.research.google.com/github/MayThiriKyaw2310/spam_mail_detection/blob/main/Spam_Email_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
!pip install numpy pandas nltk scikit-learn

In [66]:
!pip install stopwords


In [67]:
import string
import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, precision_recall_curve
from sklearn.model_selection import GridSearchCV

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [68]:
df = pd.read_csv('/content/contact_data1.csv')
df.head()


,ID,Name,Email,Details,Message,Address,label,classification
0,1,jane doe,janedoe@gmail.com,Contact for null,mike tl,NaN,0,non-spam
1,2,Richard Davidson,leadingai@dollartip.info,Contact for undefined,"Hi kalasa.gallery, Are you ready to take your ...",NaN,1,spam
2,3,Jason Groves,aiinteractivebookscommercial@growthmarketingno...,Contact for undefined,"Hey, Guess What ? Now You Launch Your Own 6-Fi...",NaN,1,spam
3,4,William Coleman,aitools@getmoreopportunities.info,Contact for undefined,"Hi, Imagine having access to the world's lead...",NaN,1,spam
4,5,Sarah Blake,sarah.blake@techinnovators.com,Contact for product inquiries,We are excited to announce the launch of our ...,NaN,1,spam


Handling Missing Values


In [69]:
print(df.isnull().values.any())
print(df.isnull().sum())

df['Address'].fillna('Unknown Address', inplace=True)
print(df)

True
ID                 0
Name               0
Email              0
Details            0
Message            0
Address           21
label              0
classification     0
dtype: int64
    ID                 Name  \
0    1             jane doe   
1    2     Richard Davidson   
2    3         Jason Groves   
3    4      William Coleman   
4    5          Sarah Blake   
5    6      George Thompson   
6    7        Emily Roberts   
7    8          David Clark   
8    9       Linda Williams   
9   10          Michael Lee   
10  11      Nathaniel Ayala   
11  12         Robert Banks   
12  13    Business Registry   
13  14        Thomas Taylor   
14  15      Clarence Anders   
15  16    Business Registry   
16  17      Candie Mitchell   
17  18          Rex Kimball   
18  19       William Felipe   
19  20          Alvin Toney   
20  21          Rocky Terry   
21  22     Jimmy Blackshire   
22  23       Carlos Spencer   
23  24        Vernon Wilson   
24  25         Tawnya Keefe   
25  26  

<ipython-input-69-4427dd3a686e>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Address'].fillna('Unknown Address', inplace=True)


In [70]:
df['Message'] = df['Message'].apply(lambda x: x.replace('\r\n', ''))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              47 non-null     int64 
 1   Name            47 non-null     object
 2   Email           47 non-null     object
 3   Details         47 non-null     object
 4   Message         47 non-null     object
 5   Address         47 non-null     object
 6   label           47 non-null     int64 
 7   classification  47 non-null     object
dtypes: int64(2), object(6)
memory usage: 3.1+ KB


In [71]:
# Defining a custom stopwords set
stopwords_set = {
    "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your",
    "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she",
    "her", "hers", "herself", "it", "its", "itself", "they", "them", "their",
    "theirs", "themselves", "what", "which", "who", "whom", "this", "that",
    "these", "those", "am", "is", "are", "was", "were", "be", "been", "being",
    "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an",
    "the", "and", "but", "if", "or", "because", "as", "until", "while", "of",
    "at", "by", "for", "with", "about", "against", "between", "into", "through",
    "during", "before", "after", "above", "below", "to", "from", "up", "down",
    "in", "out", "on", "off", "over", "under", "again", "further", "then",
    "once", "here", "there", "when", "where", "why", "how", "all", "any",
    "both", "each", "few", "more", "most", "other", "some", "such", "no",
    "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s",
    "t", "can", "will", "just", "don", "should", "now"
}

stemmer = PorterStemmer()

corpus = []

# Process each message in the DataFrame
for i in range(len(df)):
    text = df['Message'].iloc[i].lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text =text.split()
    text = [stemmer.stem(word) for word in text if word not in stopwords_set]
    text = ' '.join(text)

    corpus.append(text)

# Storing the processed messages in the DataFrame
df['Processed_Message'] = corpus

#print(df['Processed_Message'].iloc[2])


In [ ]:
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(corpus).toarray()
y = df.label

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42, stratify=y)

from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='auto',random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

import xgboost as xgb
scale_pos_weight = np.sum(y_resampled == 0) / np.sum(y_resampled == 1)
xgb_model = xgb.XGBClassifier(scale_pos_weight=len(y_resampled) / (2 * np.bincount(y_resampled)),
                              objective='binary:logistic',
                              eval_metric='logloss',
                              use_label_encoder=False)

# Set up hyperparameter grid for tuning
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [50, 100, 150],
    'scale_pos_weight': [1.0, 2.0, 3.0]
}

# Grid search to find best hyperparameters
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_resampled, y_resampled)

# Get the best model from grid search
best_xgb_model = grid_search.best_estimator_

# Evaluate the model
y_pred = best_xgb_model.predict(X_test)

# Print the classification report
print("XGBoost Classification Report:")
print(classification_report(y_test, y_pred))


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [16:11:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [16:11:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [16:11:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [16:11:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [16:11:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_e

In [ ]:
# Evaluation
y_pred = best_xgb_model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


In [ ]:
y_probs = best_xgb_model.predict_proba(X_test)[:, 1]

precision, recall, thresholds = precision_recall_curve(y_test, y_probs)

optimal_threshold = thresholds[np.argmax(2 * (recall * precision) / (recall + precision))]
print(f"Optimal Threshold: {optimal_threshold}")

#Setting the Lower Threshold
threshold = 0.3
y_pred = (y_probs >= threshold).astype(int)

In [ ]:
print("Classification Report with Lower Threshold:")
print(classification_report(y_test, y_pred))

In [ ]:
# prediction
email_to_classify = df.Processed_Message[39]
email_to_classify

In [ ]:
email_message = email_to_classify.lower().translate(str.maketrans("","",string.punctuation )).split()
email_message = [stemmer.stem(word) for word in text if word not in stopwords_set]
email_message =  ' '.join(email_message)

email_corpus = [email_message]

X_email = vectorizer.transform(email_corpus)

best_xgb_model.predict(X_email)
df.label.iloc[3]